In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import minimize

#needs blpapi to be installed
import pdblp

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

### Query data from Bloomberg

In [61]:
con = pdblp.BCon(port=8194)
con.start()

# bloomberg indices
historical_tickers = [
    'G0O1 Index',
    'LUATTRUU Index',
    'LBUTTRUU Index',
    'BCOMTR Index',
    'SPX Index',
    'BCOMGCTR Index',
    'JPEIGLBL Index',
    'LF98TRUU Index',
    'LT08TRUU Index'
]

names = [
    'RF',
    'US Tsy',
    'US Tips',
    'Commodities',
    'US Equities',
    'Gold',
    'EM Bonds',
    'HY Bonds',
    'Intermediate US Tsy'
]

flds = ['PX_LAST']
startDate = 19701231
rdf = con.bdh(historical_tickers, flds, startDate, '')

# remove multiindex
rdf.columns = rdf.columns.droplevel(1)
rdf = rdf.rename(columns=dict(zip(historical_tickers,names)))

# convert to monthly
rdf = rdf.resample('M').last()

# % change
rdf = rdf.pct_change()

con.stop()
rdf.tail()

ticker,Intermediate US Tsy,RF,US Tsy,US Tips,Commodities,US Equities,Gold,EM Bonds,HY Bonds
date,,,,,,,,,
2017-12-31,0.000271,0.001113,0.003093,0.009155,0.029855,0.009832,0.026634,0.006302,0.003025
2018-01-31,-0.009758,0.001193,-0.013589,-0.008574,0.019859,0.056179,0.023434,-0.001956,0.005995
2018-02-28,-0.002999,0.000946,-0.007534,-0.009719,-0.017295,-0.038947,-0.017578,-0.019594,-0.008498
2018-03-31,0.005320,0.001402,0.009430,0.010505,-0.006215,-0.026885,0.004056,0.003769,-0.006041
2018-04-30,-0.002769,0.000541,-0.004091,0.000615,0.021356,0.005843,0.016255,0.001248,0.009197


In [70]:
print("Start Dates")
for i,n in enumerate(names):
    null_mask = pd.isnull(rdf[n])
    print("%s : %s" % (n, str(rdf.loc[null_mask,n].index[-1].date())))

Start Dates
RF : 1977-12-31
US Tsy : 1973-01-31
US Tips : 1997-03-31
Commodities : 1991-01-31
US Equities : 1970-12-31
Gold : 1991-01-31
EM Bonds : 1993-12-31
HY Bonds : 1983-07-31
Intermediate US Tsy : 1973-01-31


### Growth & Inflation Data

In [71]:
gidf = pd.read_csv('macrodf.csv')
gidf = gidf.set_index('date')
gidf.head(15).tail(5)

,INDPRO,INDPRO Trend,dindprod3,INDPRO Monthly,INDPRO Quarterly,INDPRO Forecast Surprise,INDPRO Forecast Surprise Normalized,INDPRO Forecast Surprise Signal,INDPRO Trend Surprise,INDPRO Trend Surprise Normalized,...,CPURNSA Index Monthly,CPURNSA Index Quarterly,CPI Forecast Surprise,CPI Forecast Surprise Normalized,CPI Forecast Surprise Signal,CPI Trend Surprise,CPI Trend Surprise Normalized,CPI Trend Surprise Signal,CPI Combined Surprise Normalized,CPI Combined Surprise Signal
date,,,,,,,,,,,,,,,,,,,,,
1970-10-31,37.7154,NaN,0.048184,-0.215307,-0.014194,-0.062378,-1.039537,0.0,NaN,NaN,...,0.062972,0.062645,0.032685,0.988772,1.0,NaN,NaN,NaN,NaN,NaN
1970-11-30,37.4870,NaN,0.048184,-0.070298,-0.014194,-0.062378,-1.039537,0.0,NaN,NaN,...,0.062643,0.062645,0.032685,0.988772,1.0,NaN,NaN,NaN,NaN,NaN
1970-12-31,38.3479,NaN,0.048184,0.313200,-0.014194,-0.062378,-1.039537,0.0,NaN,NaN,...,0.062318,0.062645,0.032685,0.988772,1.0,NaN,NaN,NaN,NaN,NaN
1971-01-31,38.6429,-0.036150,0.154570,0.096320,0.018855,-0.135715,-2.261695,0.0,0.132470,1.545854,...,0.000000,0.020253,-0.016760,-0.507037,0.0,-0.055725,-1.417889,0.0,-0.962463,0.0
1971-02-28,38.5692,-0.010502,0.154570,-0.022648,0.018855,-0.135715,-2.261695,0.0,-0.012146,-0.141738,...,0.030571,0.020253,-0.016760,-0.507037,0.0,-0.022370,-0.569183,0.0,-0.538110,0.0


### Combine Growth and Inflation surprise data with monthly return data

In [72]:
rdf2 = pd.merge(
    rdf,
    gidf.loc[:,
             [
                 'INDPRO Forecast Surprise Signal',
                 'INDPRO Trend Surprise Signal',
                 'INDPRO Combined Surprise Signal',
                 'CPI Forecast Surprise Signal',
                 'CPI Trend Surprise Signal',
                 'CPI Combined Surprise Signal'
             ]
            ],
    how='left',
    left_index=True,
    right_index=True
)

# start at 73
start73_mask = rdf2.index >= pd.datetime(1973,1,1)

# end in 2017
end17_mask = rdf2.index <= pd.datetime(2017,12,31)

rdf2 = rdf2[(start73_mask & end17_mask)]
rdf2.head(12)

,Intermediate US Tsy,RF,US Tsy,US Tips,Commodities,US Equities,Gold,EM Bonds,HY Bonds,INDPRO Forecast Surprise Signal,INDPRO Trend Surprise Signal,INDPRO Combined Surprise Signal,CPI Forecast Surprise Signal,CPI Trend Surprise Signal,CPI Combined Surprise Signal
date,,,,,,,,,,,,,,,
1973-01-31,NaN,NaN,NaN,NaN,NaN,-0.017111,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,1.0
1973-02-28,-0.002728,NaN,-0.002627,NaN,NaN,-0.037490,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0
1973-03-31,-0.000101,NaN,0.000000,NaN,NaN,-0.001433,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,1.0
1973-04-30,0.009422,NaN,0.009420,NaN,NaN,-0.040800,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0
1973-05-31,0.002609,NaN,0.002208,NaN,NaN,-0.018884,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0
1973-06-30,0.002803,NaN,0.002703,NaN,NaN,-0.006575,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0
1973-07-31,-0.020463,NaN,-0.021068,NaN,NaN,0.037982,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0
1973-08-31,0.019260,NaN,0.019584,NaN,NaN,-0.036685,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,1.0
1973-09-30,0.021196,NaN,0.022009,NaN,NaN,0.040096,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


In [73]:
# convert to spread indices to remove Tsy component
rdf2['HY Spread'] = rdf2['HY Bonds'] - rdf2['Intermediate US Tsy']
rdf2['EM Spread'] = rdf2['EM Bonds'] - rdf2['Intermediate US Tsy']

In [80]:
signal_cols_dict = {
    'Forecast' : ('INDPRO Forecast Surprise Signal','CPI Forecast Surprise Signal'),
    'Trend' : ('INDPRO Trend Surprise Signal','CPI Trend Surprise Signal'),
    'Combined' : ('INDPRO Combined Surprise Signal','CPI Combined Surprise Signal')
}

cash_ticker = 'RF'

tickers = np.array(
    [
        'US Tsy',
        'US Tips',
        'US Equities',
        'Commodities',
        'Gold',
        'HY Spread',
        'EM Spread'
    ]
)

macro_environments = np.array([
                'All',
                'Growth Up',
                'Growth Down',
                'Inflation Up',
                'Inflation Down',
                'Growth Up + Inflation Up',
                'Growth Up + Inflation Down',
                'Growth Down + Inflation Up',
                'Growth Down + Inflation Down'
            ])

macro_envs_ordered = np.array([
    'Growth Down + Inflation Down',
    'Growth Up + Inflation Down',
    'Growth Down + Inflation Up',
    'Growth Up + Inflation Up'
])

In [81]:
def create_macro_environment_masks(
    rdf:pd.DataFrame,
    signal_type:str,
    sc_dict:dict=signal_cols_dict
) -> pd.DataFrame:
    
    mdf = pd.DataFrame(index=rdf.index)
    
    mdf['Growth Up'] = rdf[sc_dict[signal_type][0]] > 0.9
    mdf['Growth Down'] = rdf[sc_dict[signal_type][0]] < 0.1
    mdf['Inflation Up'] = rdf[sc_dict[signal_type][1]] > 0.9
    mdf['Inflation Down'] = rdf[sc_dict[signal_type][1]] < 0.1
    
    mdf['Growth Up + Inflation Up'] = mdf['Growth Up'] & mdf['Inflation Up']
    mdf['Growth Up + Inflation Down'] = mdf['Growth Up'] & mdf['Inflation Down']
    mdf['Growth Down + Inflation Up'] = mdf['Growth Down'] & mdf['Inflation Up']
    mdf['Growth Down + Inflation Down'] = mdf['Growth Down'] & mdf['Inflation Down']
    return mdf
    
def print_macro_environment_masks(mdf:pd.DataFrame) -> None:
    print("Rising Growth number of Months: %i" % np.nansum(mdf['Growth Up']))
    print("Rising Growth %% of Months: %.2f%%" % (np.nanmean(mdf['Growth Up'])*100))
    print("Falling Growth number of Months: %i" % np.nansum(mdf['Growth Down']))
    print("Falling Growth %% of Months: %.2f%%" % (np.nanmean(mdf['Growth Down'])*100))
    print("Rising Inflation number of Months: %i" % np.nansum(mdf['Inflation Up']))
    print("Rising Inflation %% of Months: %.2f%%" % (np.nanmean(mdf['Inflation Up'])*100))
    print("Falling Inflation number of Months: %i" % np.nansum(mdf['Inflation Down']))
    print("Falling Inflation %% of Months: %.2f%%" % (np.nanmean(mdf['Inflation Down'])*100))
    print("Rising Growth + Rising Inflation number of Months: %i" % np.nansum(mdf['Growth Up + Inflation Up']))
    print("Rising Growth + Rising Inflation %% of Months: %.2f%%" % (np.nanmean(mdf['Growth Up + Inflation Up'])*100))
    print("Rising Growth + Falling Inflation number of Months: %i" % np.nansum(mdf['Growth Up + Inflation Down']))
    print("Rising Growth + Falling Inflation %% of Months: %.2f%%" % (np.nanmean(mdf['Growth Up + Inflation Down'])*100))
    print("Falling Growth + Rising Inflation number of Months: %i" % np.nansum(mdf['Growth Down + Inflation Up']))
    print("Falling Growth + Rising Inflation %% of Months: %.2f%%" % (np.nanmean(mdf['Growth Down + Inflation Up'])*100))
    print("Falling Growth + Falling Inflation number of Months: %i" % np.nansum(mdf['Growth Down + Inflation Down']))
    print("Falling Growth + Falling Inflation %% of Months: %.2f%%" % (np.nanmean(mdf['Growth Down + Inflation Down'])*100))
    return

In [82]:
def calculate_sharpes_in_macro_envs(
    rdf:pd.DataFrame,
    mdf:pd.DataFrame,
    macro_environments:np.ndarray,
    tickers:np.ndarray,
    cash_ticker:str=cash_ticker) -> pd.DataFrame:
    
    resdf = pd.DataFrame(
        np.zeros((len(tickers),len(macro_environments))),
        index=tickers,
        columns=macro_environments
    )
    
    for t in tickers:
        #no need to subtract the risk free in self financing portfolios
        #this assumes going long HY and short duration matched treasuries is dollar neutral
        if t in ("HY Spread","EM Spread"):
            for m in macro_environments:
                if m == 'All':
                    resdf.loc[t,m] = (
                        (np.power(1+np.nanmean(rdf.loc[:,t]),12) - 1)
                        /(np.nanstd(rdf.loc[:,t])*np.sqrt(12))
                    )
                else:
                    mask = mdf[m]
                    resdf.loc[t,m] = (
                        (np.power(1+np.nanmean(rdf.loc[mask,t]),12) - 1)
                        /(np.nanstd(rdf.loc[mask,t])*np.sqrt(12))
                    )
        else:
            for m in macro_environments:
                if m == 'All':
                    resdf.loc[t,m] = (
                        (np.power(1+np.nanmean(rdf.loc[:,t] - rdf.loc[:,cash_ticker]),12) - 1)
                        /(np.nanstd(rdf.loc[:,t])*np.sqrt(12))
                    )
                else:
                    mask = mdf[m]
                    resdf.loc[t,m] = (
                        (np.power(1+np.nanmean(rdf.loc[mask,t] - rdf.loc[mask,cash_ticker]),12) - 1)
                        /(np.nanstd(rdf.loc[mask,t])*np.sqrt(12))
                    )
    return resdf

## Examine Relationships with Macro Environments

In [83]:
signal_type = 'Forecast'
print(signal_type)

mdf_fc = create_macro_environment_masks(rdf2,signal_type,sc_dict=signal_cols_dict)
print_macro_environment_masks(mdf_fc)

signal_type = 'Trend'
print()
print(signal_type)

mdf_trend = create_macro_environment_masks(rdf2,signal_type,sc_dict=signal_cols_dict)
print_macro_environment_masks(mdf_trend)

signal_type = 'Combined'
print()
print(signal_type)

mdf_combined = create_macro_environment_masks(rdf2,signal_type,sc_dict=signal_cols_dict)
print_macro_environment_masks(mdf_combined)

Forecast
Rising Growth number of Months: 237
Rising Growth % of Months: 43.89%
Falling Growth number of Months: 303
Falling Growth % of Months: 56.11%
Rising Inflation number of Months: 276
Rising Inflation % of Months: 51.11%
Falling Inflation number of Months: 264
Falling Inflation % of Months: 48.89%
Rising Growth + Rising Inflation number of Months: 120
Rising Growth + Rising Inflation % of Months: 22.22%
Rising Growth + Falling Inflation number of Months: 117
Rising Growth + Falling Inflation % of Months: 21.67%
Falling Growth + Rising Inflation number of Months: 156
Falling Growth + Rising Inflation % of Months: 28.89%
Falling Growth + Falling Inflation number of Months: 147
Falling Growth + Falling Inflation % of Months: 27.22%

Trend
Rising Growth number of Months: 252
Rising Growth % of Months: 46.67%
Falling Growth number of Months: 288
Falling Growth % of Months: 53.33%
Rising Inflation number of Months: 268
Rising Inflation % of Months: 49.63%
Falling Inflation number of Mo

In [84]:
mdf_fc.head()

,Growth Up,Growth Down,Inflation Up,Inflation Down,Growth Up + Inflation Up,Growth Up + Inflation Down,Growth Down + Inflation Up,Growth Down + Inflation Down
date,,,,,,,,
1973-01-31,True,False,True,False,True,False,False,False
1973-02-28,True,False,True,False,True,False,False,False
1973-03-31,True,False,True,False,True,False,False,False
1973-04-30,False,True,True,False,False,False,True,False
1973-05-31,False,True,True,False,False,False,True,False


In [85]:
sdf_fc = calculate_sharpes_in_macro_envs(
    rdf2,
    mdf_fc,
    macro_environments,
    tickers,
    cash_ticker
)
print("Sharpes using Forecasts to determine economic surprises")
sdf_fc

Sharpes using Forecasts to determine economic surprises


,All,Growth Up,Growth Down,Inflation Up,Inflation Down,Growth Up + Inflation Up,Growth Up + Inflation Down,Growth Down + Inflation Up,Growth Down + Inflation Down
US Tsy,0.433194,-0.274963,0.962414,-0.156239,1.055512,-0.960672,0.489157,0.457796,1.476199
US Tips,0.606293,0.714853,0.582368,1.097410,0.187884,0.992488,0.464139,1.143656,0.106190
US Equities,0.311140,0.529214,0.149316,0.015537,0.598534,0.221331,0.811413,-0.127496,0.431634
Commodities,0.048666,0.196881,-0.031586,0.979920,-0.576675,1.084529,-0.345127,0.944769,-0.730631
Gold,0.199295,-0.173360,0.403444,0.604647,-0.104348,0.144793,-0.386472,0.848778,0.049390
HY Spread,0.336426,0.703838,0.172872,0.522841,0.205489,0.695229,0.711698,0.461172,-0.037501
EM Spread,0.437417,0.482090,0.407102,0.591079,0.346722,0.564218,0.442431,0.639761,0.243800


In [86]:
sdf_trend = calculate_sharpes_in_macro_envs(
    rdf2,
    mdf_trend,
    macro_environments,
    tickers,
    cash_ticker
)
print("Sharpes using Trend to determine economic surprises")
sdf_trend

Sharpes using Trend to determine economic surprises


,All,Growth Up,Growth Down,Inflation Up,Inflation Down,Growth Up + Inflation Up,Growth Up + Inflation Down,Growth Down + Inflation Up,Growth Down + Inflation Down
US Tsy,0.433194,-0.099936,0.878198,-0.031508,0.841731,-0.250665,0.047433,0.175038,1.483363
US Tips,0.606293,0.406265,0.815919,0.986297,0.247912,1.281588,-0.368610,0.718701,0.920867
US Equities,0.311140,0.141814,0.470395,0.164697,0.456626,0.081057,0.198032,0.237638,0.718185
Commodities,0.048666,0.138962,-0.028703,0.789510,-0.488887,1.166839,-0.636610,0.460244,-0.372845
Gold,0.199295,-0.048192,0.453124,0.557704,-0.126703,0.460321,-0.525227,0.659306,0.268066
HY Spread,0.336426,0.577839,0.117317,0.552841,0.155595,1.017938,0.227807,0.151668,0.088139
EM Spread,0.437417,0.169522,0.814970,0.559602,0.339416,0.234291,0.128624,0.899233,0.723154


In [87]:
sdf_combined = calculate_sharpes_in_macro_envs(
    rdf2,
    mdf_combined,
    macro_environments,
    tickers,
    cash_ticker
)
print("Sharpes using a combination of Forecasts and Trend to determine economic surprises")
sdf_combined

Sharpes using a combination of Forecasts and Trend to determine economic surprises


,All,Growth Up,Growth Down,Inflation Up,Inflation Down,Growth Up + Inflation Up,Growth Up + Inflation Down,Growth Down + Inflation Up,Growth Down + Inflation Down
US Tsy,0.433194,-0.134944,0.840712,-0.072072,1.000282,-0.323310,0.085777,0.111057,1.688361
US Tips,0.606293,0.702030,0.539791,0.855499,0.358652,1.679673,0.013137,0.438838,0.666787
US Equities,0.311140,0.361782,0.270189,0.055378,0.572870,0.190835,0.508155,-0.034782,0.639738
Commodities,0.048666,0.244555,-0.092531,0.872933,-0.525555,1.452471,-0.460892,0.529455,-0.578387
Gold,0.199295,-0.038349,0.390234,0.595648,-0.123290,0.470854,-0.358310,0.676913,0.106299
HY Spread,0.336426,0.495175,0.230777,0.488331,0.209428,0.832720,0.279175,0.309517,0.153996
EM Spread,0.437417,0.276930,0.637504,0.525999,0.377901,0.700022,0.066646,0.405526,0.920460
